In [165]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [166]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth', 'Utils', 'Other', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = '10 Industry Portfolios - Average Value Weighted Returns.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [167]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    rf = pd.read_csv(file_name, delimiter = ',', header = 0, usecols = ['RF'])
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = data

actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
date,,,,,,,,,,,
196307,-0.47,-0.22,-1.41,2.29,-0.69,-0.23,-1.03,0.56,0.80,-1.61,-0.12
196308,4.87,6.55,6.20,3.93,5.14,4.29,6.40,9.56,4.20,5.49,5.32
196309,-1.68,-0.24,-0.76,-3.64,0.13,2.36,0.96,-4.06,-2.50,-3.16,-1.30
196310,2.66,9.72,2.58,-0.32,8.29,3.40,0.48,3.38,-0.67,1.38,2.82
196311,-1.13,-4.84,0.30,-1.15,-0.29,4.16,-1.23,-1.65,-1.02,0.23,-0.58


In [168]:
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

In [169]:
# simulerer 1/N portfølje fra 1963-07 til 2004-11 som rollover med 60 måneders interval:
for month in range(0, len(returns_data) - 60):
    
    # Danner nyt data interval, returns og cov-matrix for hvert interval
    new_returns_data = returns_data[month:month + 60]
    returns_monthly = new_returns_data
    returns_mean = returns_monthly.mean()
    cov_matrix = returns_monthly.cov()
    
    # udregner profit, volatilitet og sharpe
    tl = np.dot(np.linalg.inv(cov_matrix), np.ones(11))
    nl = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), np.ones(11)))
    weights = tl / nl
    
    # udregner afkast for måned T + 1 med de udregnede optimale vægte
    returns = np.dot(weights, returns_data[month + 60:month + 61].mean()) - float(rf.mean())
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = returns / volatility

    # indsætter overstående udregninger i vektorerne
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

In [170]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

In [171]:
# definerer de enkelte vægte af de valgte aktiver, HINT: de er 0.33 alle sammen da det er 1/N
for i in range(len((actives))):
    symbol = actives[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [172]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in actives]
df = df[column_order]

df.head() # Porteføljer ud fra 60 måneders interval
          # ie. portfølje '0' har return, vol og sharpe ud fra måned 1 - 60
          # portfølje '1' har således overstående ud fra måned 2 - 61 ovs.

,Returns,Volatility,Sharpe Ratio,NoDur Weight,Durbl Weight,Manuf Weight,Enrgy Weight,HiTec Weight,Telcm Weight,Shops Weight,Hlth Weight,Utils Weight,Other Weight,Mkt Weight
0,0.549206,2.169135,0.253191,0.548040,-0.003496,0.695398,0.734670,0.397748,0.444581,0.247967,-0.062014,0.428585,-0.103708,-2.327769
1,0.810418,2.168867,0.373660,0.549781,-0.004395,0.700911,0.729212,0.398885,0.444082,0.250392,-0.062711,0.431545,-0.103710,-2.333991
2,0.856927,2.133148,0.401719,0.513361,0.046486,0.851239,0.798510,0.419551,0.489615,0.308170,0.031680,0.459503,-0.081552,-2.836562
3,0.359893,2.128219,0.169105,0.542333,0.047425,0.863164,0.825234,0.428304,0.480773,0.302439,0.054417,0.467430,-0.082820,-2.928699
4,6.265689,2.128039,2.944349,0.539394,0.041111,0.866746,0.826598,0.423768,0.477801,0.306305,0.054618,0.466510,-0.083295,-2.919555


In [173]:
df['Returns'].mean() / df['Returns'].std()

0.15376077747169

In [174]:
len(df)

437

In [175]:
df['Sharpe Ratio'].mean()

0.21459836652687278